In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [2]:
# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('./data/train.txt')

In [3]:
# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)

print('샘플 크기 : \n', len(sentences))
print('0번째 샘플 문장 시퀀스 : \n', sentences[0])
print('0번째 샘플 bio 태그 : \n', tags[0])
print('샘플 문장 시퀀스 최대 길이 : \n', max(len(l) for  l in sentences))
print('샘플 문장 시퀀스 평균 길이 : \n', (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 3555
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 : 
 168
샘플 문장 시퀀스 평균 길이 : 
 34.03909985935302


In [4]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') # 첫 번째 인덱스에는 OOV 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

# 단어 사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print('BIO 태그 사전 크기 : ', tag_size)
print('단어 사전 크기 : ', vocab_size)

BIO 태그 사전 크기 :  8
단어 사전 크기 :  13834


In [5]:
print(sent_tokenizer.word_index)
print(tag_tokenizer.word_index)

{'OOV': 1, '하': 2, '.': 3, '이': 4, '을': 5, '는': 6, 'ㄴ': 7, '다': 8, '의': 9, '에': 10, ',': 11, '를': 12, '은': 13, '았': 14, '고': 15, '(': 16, ')': 17, '었': 18, '가': 19, '아': 20, '에서': 21, '으로': 22, '있': 23, '일': 24, '되': 25, '로': 26, "'": 27, '어': 28, 'ㄹ': 29, '과': 30, '들': 31, '1': 32, '2': 33, '도': 34, '와': 35, '-': 36, '등': 37, '것': 38, '지': 39, '3': 40, 'ㄴ다': 41, '년': 42, '"': 43, '월': 44, '%': 45, '만': 46, '적': 47, '기': 48, '게': 49, '4': 50, '시': 51, '원': 52, '며': 53, '수': 54, '5': 55, '밝히': 56, '대하': 57, '주': 58, '말': 59, '다고': 60, '면': 61, '던': 62, '위하': 63, '전': 64, '오': 65, '까지': 66, '10': 67, '대': 68, '지나': 69, '부터': 70, '7': 71, ':': 72, '6': 73, '한국': 74, '받': 75, '않': 76, '명': 77, '경기': 78, '대표': 79, '보': 80, '지만': 81, '라고': 82, '시장': 83, '에게': 84, '따르': 85, '9': 86, '8': 87, '개': 88, '내': 89, '중': 90, '서울': 91, '‘': 92, '분': 93, '억': 94, '회': 95, '’': 96, '감독': 97, '의원': 98, '기록': 99, '제': 100, '~': 101, '11': 102, '습니다': 103, '팀': 104, '그': 105, 'ㄴ다고': 106, '이번': 107, '면서': 

In [6]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(sentences[0])
print(x_train[0])
print(y_train[0])

# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word   # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word     # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
# 시퀀스 패딩 처리 전
print(x_train)
print(y_train)

[[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3], [1910, 42, 73, 44, 231, 24, 2609, 430, 64, 206, 4277, 84, 513, 14, 62, 67, 748, 6348, 5, 1517, 55, 42, 46, 10, 3252, 6, 932, 23, 6, 275, 4, 18, 8, 3], [3253, 532, 13, 87, 24, 16, 320, 74, 135, 17, 6349, 11, 6350, 5, 514, 2, 20, 6351, 6352, 11, 6353, 6354, 97, 11, 4278, 6355, 97, 11, 321, 6356, 6357, 6358, 11, 2198, 6359, 9, 4279, 4280, 6360, 37, 4, 184, 1070, 9, 1681, 6361, 191, 26, 383, 25, 18, 60, 80, 990, 2, 14, 8, 3], [16, 1249, 73, 748, 17, 43, 3254, 6362, 2198, 97, 13, 3255, 21, 4281, 6, 6363, 4, 108, 6364, 4, 8, 3], [608, 2199, 3256, 7, 6365, 6366, 6, 73, 24, 78, 21, 1911, 5, 515, 34, 454, 39, 76, 15, 2200, 71, 88, 12, 1362, 6, 6367, 6368, 5, 2, 18, 8, 3], [6369, 97, 13, 87, 24, 16, 320, 74, 135, 17, 43, 125, 6, 533, 192, 19, 1250, 162, 1149, 4, 25, 29, 54, 23, 5, 38, 4, 82, 301, 2, 41, 3], [170, 3257, 828, 4282, 4282, 176, 99, 22, 80, 7, 430, 30, 210, 9, 6370, 2, 7, 2201], [308, 309,

In [8]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)
print(x_train)
print('='*45)
print(y_train)

[[  183    11  4276 ...     0     0     0]
 [ 1910    42    73 ...     0     0     0]
 [ 6352    11  6353 ...    14     8     3]
 ...
 [  387  1820    13 ...     0     0     0]
 [  531    16 13829 ...     0     0     0]
 [13831   398   451 ...     3     0     0]]
[[1 1 1 ... 0 0 0]
 [5 2 2 ... 0 0 0]
 [2 1 4 ... 1 1 1]
 ...
 [3 2 1 ... 0 0 0]
 [3 1 1 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


In [9]:
# 학습 데이터와 테스트 테이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)
print('학습 샘플 시퀀스 형상 : ', x_train.shape)
print('학습 샘플 레이블 형상 : ', y_train.shape)
print('테스트 샘플 시퀀스 형상 : ', x_test.shape)
print('테스트 샘플 레이블 형상 : ', x_test.shape)

학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 8)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40)


In [45]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10 )

print('평가 결과 : ', model.evaluate(x_test, y_test)[1])

# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequenc in sequences:
        temp = []
        for pred in sequenc:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace('PAD', 'O'))
        result.append(temp)
    return result

Epoch 1/10
23/23 [==============================] - 18s 589ms/step - loss: 0.6956 - accuracy: 0.8346
Epoch 2/10
23/23 [==============================] - 13s 583ms/step - loss: 0.3075 - accuracy: 0.9029
Epoch 3/10
23/23 [==============================] - 14s 587ms/step - loss: 0.2193 - accuracy: 0.9245
Epoch 4/10
23/23 [==============================] - 13s 585ms/step - loss: 0.1655 - accuracy: 0.9442
Epoch 5/10
23/23 [==============================] - 13s 582ms/step - loss: 0.1215 - accuracy: 0.9612
Epoch 6/10
23/23 [==============================] - 13s 578ms/step - loss: 0.0831 - accuracy: 0.9745
Epoch 7/10
23/23 [==============================] - 14s 589ms/step - loss: 0.0614 - accuracy: 0.9813
Epoch 8/10
23/23 [==============================] - 13s 586ms/step - loss: 0.0482 - accuracy: 0.9851
Epoch 9/10
23/23 [==============================] - 14s 594ms/step - loss: 0.0399 - accuracy: 0.9869
Epoch 10/10
23/23 [==============================] - 3s 101ms/step - loss: 0.2812 - accurac

In [46]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test) # (711, 40) => model => (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted)   # 예측된 NER
test_tags = sequences_to_tag(y_test)        # 실제 NER

# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print('F1-score : {:.1%}'.format(f1_score(test_tags,pred_tags)))

23/23 [==============================] - 3s 100ms/step


c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.60      0.62      0.61       657
         _DT       0.88      0.90      0.89       335
         _LC       0.75      0.52      0.62       312
         _OG       0.66      0.56      0.61       481
         _PS       0.81      0.42      0.55       374
         _TI       0.87      0.73      0.79        66

   micro avg       0.71      0.60      0.65      2225
   macro avg       0.76      0.62      0.68      2225
weighted avg       0.72      0.60      0.65      2225

F1-score : 65.3%


In [47]:
# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자가 2023년 6월에 출시한 갤럭시24 스마트폰이 전년 대비 두 배의 판매 실적을 보인다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w,1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])

print('새로운 유형의 시퀀스 : ', new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0, maxlen=max_len)

# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1)   # 예픅된 NER 인덱스 값 추출

print('{:10}    {:5}'.format('단어', '예측된 NER'))
print('-'*50)
for w, pred in zip(new_sentence, p[0]):
    print('{:10}    {:5}'.format(w, index_to_ner[pred]))

새로운 유형의 시퀀스 :  [1, 1, 1, 1, 1, 1, 827, 377, 133, 1, 371, 1, 1]
1/1 [==============================] - 0s 39ms/step
단어            예측된 NER
--------------------------------------------------
삼성전자가         O    
2023년         O    
6월에           O    
출시한           O    
갤럭시24         O    
스마트폰이         I    
전년            B_DT 
대비            O    
두             O    
배의            O    
판매            O    
실적을           O    
보인다.          O    
